# **Fetch The API Keys**

In [1]:
from google.colab import userdata

PINECONE_API_KEY=userdata.get("PINECONE_API_KEY")
GOOGLE_API_KEY=userdata.get("GOOGLE_API_KEY")
LANG_API_KEY=userdata.get("LANG_API_KEY")
OPENAI_API_KEY=userdata.get("OPENAI_KEY")

if PINECONE_API_KEY:
  print("API key fetched")
else:
  print("API key not fetched")

API key fetched


# **Initialize Pinecone with your API key**

In [4]:
from pinecone import Pinecone

pc=Pinecone(api_key=PINECONE_API_KEY)


# **Create a new Pinecone Index**

In [6]:
from pinecone import ServerlessSpec

index_name = "project-2-rag-wtih-langchain-v1"

pc.create_index(
  name=index_name,
  dimension=768,
  spec=ServerlessSpec(cloud='aws',region='us-east-1')
)

# **Initialize the Index**

In [7]:
index = pc.Index(index_name)


# **Check the current index statistics**

In [8]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

# **Initialize Google Generative AI embeddings with the API key and model name**

In [10]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(google_api_key=GOOGLE_API_KEY,model = "models/embedding-001")

# **Embed the query "Tell me my full name"**

In [11]:
vector= embeddings.embed_query("Tell me my full name")

# **Initialize the Pinecone vector store with the index and embeddings**

In [13]:
from langchain_pinecone import PineconeVectorStore

vector_store=PineconeVectorStore(index=index,embedding=embeddings)

# **Read and create Chunks from the input file**

In [14]:
file_path = "/content/MyData.txt"

with open(file_path, "r") as file:
    document_text = file.read()

chunks = document_text.split('\n')
print(f"Total number of chunks: {len(chunks)}")

Total number of chunks: 8


# **Creating Document Objects from Text Chunks**

In [15]:
from langchain_core.documents import Document
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(chunks))]
documents = [
    Document(
        page_content=chunk,
        metadata={"source": "documents.txt", "chunk_index": idx},
    )
    for idx, chunk in enumerate(chunks)
]

In [16]:
for doc in documents:
  print(doc.page_content)

Full Name: Muhammad Abdul Wasay Abid
Age: 20 years old
Date of Birth: May 26, 2004
Father's Name: Hafiz Abid
Brother's Name: Uzair Abid
Current University: University of Central Punjab
Current Semester: 5th semester (3rd year)
Expected Graduation Year: 2026


In [17]:
vector_store.add_documents(documents=documents,ids=uuids)

['21af278b-2fda-4534-871b-2bd72f0b385b',
 '81cd979f-f9fb-44f3-8e1b-39f1f431d813',
 '9ccb638e-7749-4801-a54e-c9f8f4f1cb2d',
 '63d3f41f-21d8-46fc-98cc-0389a736d31b',
 '08d3e26c-4c95-4876-8f85-027bcfa99b50',
 'fa687319-60ff-4f62-9d73-9688248b4277',
 '2b6352f4-ad9b-4834-a155-ff3cbc0f551e',
 '97469c53-5615-4f0d-932f-9e477d3cc0ef']

In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(
model='gemini-1.5-flash',
temperature=0.9
)


In [19]:
def answer(question:str):
  vector_results=vector_store.similarity_search(question,k=2)

  final_ans=llm.invoke(f"Answer this query :{question}, Here are some references answer {vector_results}" )
  return final_ans




In [21]:
import os
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')


In [ ]:
res=answer("what is my birthdate?")
print(res.content)

ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7c3e6d716080>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 128, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 101, in _retrieve_info
    info = _metadata.get_service_account_info(
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 323, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 248, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Go

In [198]:
from IPython.display import Markdown
Markdown(res.content)

Based on the provided text, there is conflicting information.  One document states a date of birth of May 26, 2004.  The other document states the age is 20 years old.  If the age statement is accurate and refers to the current year, the birthdate would be around 2003 or 2004.  However, only the May 26, 2004 date is explicitly given.  Therefore, the most precise answer that can be provided from the given sources is **May 26, 2004**.
